In [3]:
import pandas as pd
import numpy as np

import nltk
from nltk.tokenize import word_tokenize
from nltk import PorterStemmer
from nltk import WordNetLemmatizer
from nltk.corpus import stopwords

import re

from sklearn.feature_extraction import text
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

from sklearn import metrics
from sklearn.metrics import mean_squared_error
from sklearn.svm import SVR

In [6]:
cd

/Users/brianrochford


In [53]:
df = pd.read_excel('/Users/brianrochford/formatch.xlsx', index_col = 0)
#df.countyGroup = df.countyGroup.replace(0, np.nan)
#df = df.dropna().drop_duplicates()\
df

,description,points,price,winery,countyGroup
1,"This is ripe and fruity, a wine that is smooth...",87,15,Quinta dos Avidagos,0
2,"Tart and snappy, the flavors of lime flesh and...",87,14,Rainstorm,6
3,"Pineapple rind, lemon pith and orange blossom ...",87,13,St. Julian,0
4,"Much like the regular bottling from 2012, this...",87,65,Sweet Cheeks,6
5,Blackberry and raspberry aromas show a typical...,87,15,Tandem,2
6,"Here's a bright, informal red that opens with ...",87,16,Terre di Giurfo,0
7,This dry and restrained wine offers spice in p...,87,24,Trimbach,0
8,Savory dried thyme notes accent sunnier flavor...,87,12,Heinz Eifel,0
9,This has great depth of flavor with its fresh ...,87,27,Jean-Baptiste Adam,0
10,"Soft, supple plum envelopes an oaky structure ...",87,19,Kirkland Signature,1


In [8]:
df.countyGroup = df.countyGroup.replace(0, np.nan)

In [9]:
df = df.dropna().drop_duplicates()

In [10]:
Q1 = np.percentile(df['price'], 25,
                   interpolation = 'midpoint')
 
Q3 = np.percentile(df['price'], 75,
                   interpolation = 'midpoint')
IQR = Q3 - Q1
df = df[df.price < (Q3+1.5*IQR)+1]

Q1 = np.percentile(df['points'], 25,
                   interpolation = 'midpoint')
 
Q3 = np.percentile(df['points'], 75,
                   interpolation = 'midpoint')
IQR = Q3 - Q1
df = df[df.points < (Q3+1.5*IQR)+1]

In [65]:
"""
data1 = pd.DataFrame(df.winery.value_counts())
data1.columns = ['counts'] 

data2 = df.groupby('winery').agg({'description':lambda x: ' '.join(x), 
                         'points':'mean', 
                         'price':'mean'})
df = data1.join(data2)
"""

"\ndata1 = pd.DataFrame(df.winery.value_counts())\ndata1.columns = ['counts'] \n\ndata2 = df.groupby('winery').agg({'description':lambda x: ' '.join(x), \n                         'points':'mean', \n                         'price':'mean'})\ndf = data1.join(data2)\n"

In [11]:
remove_non_alphabets = lambda x: re.sub(r'[^a-zA-Z]',' ',x)

tokenize = lambda x: word_tokenize(x)

ps = PorterStemmer()

stem = lambda w: [ ps.stem(x) for x in w ]

lemmatizer = WordNetLemmatizer()
leammtizer = lambda x: [ lemmatizer.lemmatize(word) for word in x ]

remove_stop = lambda x: [w for w in x if w not in stop]

print('Processing : [=', end='')
df['description'] = df['description'].apply(remove_non_alphabets)
print('=', end='')
df['description'] = df['description'].apply(tokenize)
print('=', end='')
stop = text.ENGLISH_STOP_WORDS
df['description'] = df['description'].apply(remove_stop)
print('=', end='')
df['description'] = df['description'].apply(stem)
print('=', end='')
df['description'] = df['description'].apply(leammtizer)
print('=', end='')
df['description'] = df['description'].apply(lambda x: ' '.join(x))
print(']')

Processing : [======]


In [12]:
def get_sample(df,x):
    df = df.sample(frac=x, replace=False, random_state=1)
    df = df.reset_index(drop=True)
    return df

def tfidf_vector(df):
    
    tfidf_vectorizer=TfidfVectorizer(min_df=1, 
                                 norm='l2',
                                 smooth_idf=True,
                                 use_idf=True,
                                 ngram_range=(1,1),
                                     max_features=1000)
    
    tfidf_features = tfidf_vectorizer.fit_transform(df.description)  
    
    df_tfidf = pd.DataFrame(tfidf_features.toarray(),
                      columns=tfidf_vectorizer.get_feature_names())
    df_tfidf 
    
    return df_tfidf

def jointwo(df,df_tfidf):
    df = df.join(df_tfidf)
    df = df.drop(columns=['country','description'])
    df = df.fillna(0)
    return df

def run_random_clf(df):
    X_train, X_test, y_train, y_test = train_test_split(df.drop(columns=['countyGroup']), df.countyGroup, test_size=0.30, random_state=42)
    clf = RandomForestClassifier(n_estimators=100)
    clf.fit(X_train, y_train)
    return clf, X_test, y_test
    
def run_log(df):
    X_train, X_test, y_train, y_test = train_test_split(df.drop(columns=['countyGroup']), df.countyGroup, test_size=0.30, random_state=42)
    clf = LogisticRegression(random_state=0)
    clf.fit(X_train, y_train)
    return clf, X_test, y_test
    

def run_svm(df):
    X_train, X_test, y_train, y_test = train_test_split(df.drop(columns=['countyGroup']), df.countyGroup, test_size=0.30, random_state=42)
    clf = svm.SVC()
    clf.fit(X_train, y_train)
    return clf, X_test, y_test

In [13]:
df_tfidf = tfidf_vector(df.reset_index())

In [45]:
df_tfidf

,absolut,abund,accent,access,accompani,acid,ad,add,addit,afford,...,yellow,yet,yield,you,young,youth,zest,zesti,zin,zinfandel
0,0.000000,0.0,0.000000,0.000000,0.0,0.134398,0.000000,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
1,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
2,0.000000,0.0,0.000000,0.000000,0.0,0.122808,0.000000,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
3,0.000000,0.0,0.000000,0.295246,0.0,0.000000,0.000000,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
4,0.000000,0.0,0.248969,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
5,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
6,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
7,0.000000,0.0,0.000000,0.000000,0.0,0.111885,0.000000,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
8,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
9,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0


In [14]:
df_tfidf.rename(columns={'price':'pric'},inplace= True)

In [15]:
df_fin = df.reset_index().join(df_tfidf)

In [16]:
cols_to_norm = ['points','price']
df_fin[cols_to_norm] = df_fin[cols_to_norm].apply(lambda x: (x - x.min()) / (x.max() - x.min()))

In [17]:
df_ml = df_fin.iloc[:,2:].drop(columns = ['winery'])

In [26]:
df_tfidf.describe()

,absolut,abund,accent,access,accompani,acid,ad,add,addit,afford,...,yellow,yet,yield,you,young,youth,zest,zesti,zin,zinfandel
count,61299.000000,61299.000000,61299.000000,61299.000000,61299.000000,61299.000000,61299.000000,61299.000000,61299.000000,61299.000000,...,61299.000000,61299.000000,61299.000000,61299.000000,61299.000000,61299.000000,61299.000000,61299.000000,61299.000000,61299.000000
mean,0.001108,0.001653,0.006749,0.001551,0.001535,0.031655,0.002394,0.005790,0.002780,0.001493,...,0.002460,0.001864,0.001214,0.002819,0.007464,0.002761,0.002880,0.003327,0.005974,0.005657
std,0.018389,0.023063,0.038842,0.021679,0.022195,0.055733,0.025568,0.036176,0.027060,0.022166,...,0.026635,0.023384,0.019105,0.027971,0.039897,0.027228,0.027728,0.030289,0.040429,0.037582
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,0.000000,0.000000,0.000000,0.000000,0.086978,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,0.516240,0.524004,0.492757,0.506802,0.556865,0.416441,0.530937,0.548672,0.494079,0.542587,...,0.613499,0.441172,0.515133,0.472900,0.579868,0.490761,0.494914,0.490748,0.700433,0.562317


In [50]:
df_ml.to_csv('normalizedWineRegionDummy.csv')

In [41]:
groupIDF = df_ml.groupby(['countyGroup', df_ml.apply(np.mean)])
groupIDF

In [44]:
idf_byRegion = df_ml.groupby('countyGroup').agg(np.mean)
idf_byRegion.to_csv('idfByRegion.csv')

In [34]:
sorted_freq2 = df_tfidf.apply(np.mean)


In [35]:
sorted_freq2

absolut      0.001108
abund        0.001653
accent       0.006749
access       0.001551
accompani    0.001535
acid         0.031655
ad           0.002394
add          0.005790
addit        0.002780
afford       0.001493
age          0.012208
ageabl       0.001477
ager         0.001798
aggress      0.001236
air          0.001156
alcohol      0.010863
all          0.001642
allow        0.001458
almond       0.002607
alongsid     0.002348
although     0.001927
american     0.001677
amount       0.001480
ampl         0.001594
an           0.004762
ani          0.005581
appeal       0.006757
appel        0.004062
appet        0.001471
appl         0.015640
               ...   
while        0.001878
white        0.011840
wide         0.001842
wild         0.006107
will         0.001103
wine         0.048799
winemak      0.004579
wineri       0.007130
with         0.007008
wonder       0.002996
wood         0.004094
woodi        0.001493
work         0.001978
world        0.001190
worth     

In [ ]:
df_filter_rarewords = df_ml[((df_ml['region_1'] == 'Napa') |

In [ ]:
df_ml.pivot_table(index = )

In [23]:
df_ml

,points,price,countyGroup,absolut,abund,accent,access,accompani,acid,ad,...,yellow,yet,yield,you,young,youth,zest,zesti,zin,zinfandel
0,0.368421,0.126582,6.0,0.000000,0.0,0.000000,0.000000,0.0,0.134398,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
1,0.368421,0.772152,6.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
2,0.368421,0.139241,2.0,0.000000,0.0,0.000000,0.000000,0.0,0.122808,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
3,0.368421,0.189873,1.0,0.000000,0.0,0.000000,0.295246,0.0,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
4,0.368421,0.379747,2.0,0.000000,0.0,0.248969,0.000000,0.0,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
5,0.368421,0.101266,3.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
6,0.368421,0.202532,6.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
7,0.368421,0.227848,3.0,0.000000,0.0,0.000000,0.000000,0.0,0.111885,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
8,0.368421,0.822785,2.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
9,0.315789,0.151899,4.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0


## ml

In [18]:
clf, X_test, y_test = run_random_clf(df_ml)
y_pred = clf.predict(X_test)
print(metrics.classification_report(y_test,y_pred))

              precision    recall  f1-score   support

         1.0       0.58      0.29      0.38      2655
         2.0       0.45      0.68      0.54      4445
         3.0       0.58      0.49      0.53      3965
         4.0       0.54      0.45      0.49      2471
         5.0       0.68      0.85      0.76      3385
         6.0       0.78      0.31      0.44      1469

    accuracy                           0.55     18390
   macro avg       0.60      0.51      0.52     18390
weighted avg       0.58      0.55      0.54     18390



In [ ]:
feature_impRegion = pd.(feature_imp, )

In [51]:
df_feature_imp = pd.DataFrame(feature_imp)

In [52]:
df_feature_imp.to_csv('featureImportancePredictRegion.csv')

In [22]:
clf

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=0, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [ ]:
feature_imp = pd.Series(clf.feature_importances_, ,index=X_test.columns).sort_values(ascending=False)
pd.DataFrame(feature_imp)

In [19]:
feature_imp = pd.Series(clf.feature_importances_,index=X_test.columns).sort_values(ascending=False)
pd.DataFrame(feature_imp)

,0
price,0.033343
fruit,0.018573
points,0.017094
flavor,0.015320
wine,0.012572
nose,0.012247
aroma,0.011988
the,0.009213
it,0.008933
cherri,0.008807


In [20]:
clf, X_test, y_test = run_log(df_ml)
y_pred = clf.predict(X_test)
print(metrics.classification_report(y_test,y_pred))

/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


              precision    recall  f1-score   support

         1.0       0.52      0.34      0.41      2655
         2.0       0.48      0.58      0.53      4445
         3.0       0.55      0.54      0.55      3965
         4.0       0.50      0.50      0.50      2471
         5.0       0.77      0.83      0.80      3385
         6.0       0.68      0.58      0.63      1469

    accuracy                           0.57     18390
   macro avg       0.58      0.56      0.57     18390
weighted avg       0.57      0.57      0.57     18390

